<a href="https://colab.research.google.com/github/stepbot/multiScaleEvolutionarySearch/blob/master/Meta_Evolving_ML_EA_training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Meta-Evolving Genetic Algorithm Operators with LLMs (V2)

This script implements a meta-evolutionary algorithm where the core evolutionary
operator itself is evolved by an LLM.

Instead of evolving separate 'crossover' and 'mutate' functions, we evolve a
single, holistic function: `generate_next_population`. This function takes the
current population and their fitness scores and is responsible for producing
the entire next generation.

This approach gives the LLM maximum creative freedom to discover novel
evolutionary strategies, potentially departing from traditional GA structures.
"""

# @title 1. Install Dependencies
!pip install -q -U google-generativeai openai tqdm matplotlib

# @title 2. Setup and Imports
import google.generativeai as genai
import openai
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import random
import textwrap
from IPython.display import Markdown, display
import itertools
import os
import traceback

# @title 3. Configure LLM APIs
# @markdown Add your API keys to the Colab secrets manager (key icon on the left).
# @markdown - `GOOGLE_API_KEY` for Gemini
# @markdown - `OPENAI_API_KEY` for OpenAI
# @markdown The script will use any and all configured clients.

llm_clients = []

try:
    from google.colab import userdata
    # Configure Gemini
    GOOGLE_API_KEY = userdata.get('gemini_key')
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        # Use the updated model name here
        llm_clients.append(genai.GenerativeModel('gemini-2.5-pro'))
        print("✅ Successfully configured and added Gemini client.")

        # --- Debug: List available Gemini models ---
        print("\n--- Available Gemini Models (for 'generateContent') ---")
        # The genai.list_models() function gets all models
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(m.name)
        print("-----------------------------------------------------\n")
        # --- End Debug ---

    else:
        print("⚠️ Warning: GOOGLE_API_KEY not found in Colab secrets. Gemini client will not be used.")

    # Configure OpenAI
    OPENAI_API_KEY = userdata.get('openai_key')
    if OPENAI_API_KEY:
        # It's good practice to set the env variable for some libraries
        os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
        openai_client = openai.OpenAI() # Create a client instance first
        llm_clients.append(openai_client)
        print("✅ Successfully configured and added OpenAI client.")

        # --- Debug: List available OpenAI models ---
        print("\n--- Available OpenAI Models (GPT models) ---")
        try:
            # The client.models.list() function gets all models
            model_list = [m.id for m in openai_client.models.list() if 'gpt' in m.id.lower()]
            for model_name in sorted(model_list):
                print(model_name)
        except Exception as e:
            print(f"Could not retrieve OpenAI models: {e}")
        print("-------------------------------------------\n")
        # --- End Debug ---

    else:
        print("⚠️ Warning: OPENAI_API_KEY not found in Colab secrets. OpenAI client will not be used.")

except ImportError:
    print("❌ Could not import userdata. Please run this in a Google Colab environment and add API keys to secrets.")

def call_llm_api(client, prompt, model_name_gemini='gemini-2.5-pro', model_name_openai='gpt-5'):
    """A unified function to call either Gemini or OpenAI API."""
    client_name = client.__class__.__module__.split('.')[0]
    if client_name == 'google':
        model = genai.GenerativeModel(model_name_gemini)
        response = model.generate_content(prompt)
        return response.text
    elif client_name == 'openai':
        response = client.chat.completions.create(
            model=model_name_openai,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that only returns Python code."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    else:
        raise ValueError(f"Unknown client type: {client_name}")


# @title 4. Define the Machine Learning Problem (Inner Loop)

# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = 28 * 28
NUM_CLASSES = 10
BATCH_SIZE = 1024

# --- Model Architecture ---
class SimpleNet(nn.Module):
    """A simple MLP suitable for MNIST, used in the inner GA."""
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, NUM_CLASSES)

    def forward(self, x):
        x = x.view(-1, INPUT_SIZE)
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# --- Data Loading & Evaluation ---
def get_data_loaders():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

def evaluate_nn_model(model, loader):
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


# @title 5. Define the Meta-Evolutionary Algorithm (Outer Loop)

class OperatorIndividual:
    """
    Represents a single evolutionary operator, now with detailed,
    multi-run, per-generation performance history.
    """
    def __init__(self, evolution_operator_code):
        self.evolution_operator_code = evolution_operator_code
        self.run_histories = [] # Stores detailed history from multiple runs
        self.fitness = 0.0      # A single representative score for sorting/selection

    def update_history(self, new_run_histories):
        """Adds new run histories and recalculates the primary fitness score."""
        if not new_run_histories:
            return
        self.run_histories.extend(new_run_histories)
        self._calculate_primary_fitness()

    def _calculate_primary_fitness(self):
        """
        Calculates a single fitness score for elitism and selection.
        We define this as the average of the 'best' fitness from the
        final generation of each run. This rewards operators that
        consistently produce high-performing individuals.
        """
        if not self.run_histories:
            self.fitness = 0.0
            return

        final_bests = [run[-1]['best'] for run in self.run_histories if run]
        if final_bests:
            self.fitness = sum(final_bests) / len(final_bests)
        else:
            self.fitness = 0.0

    def get_performance_summary_text(self):
        """
        Processes the detailed run histories into a clean, averaged,
        human-readable summary table for the LLM prompt.
        """
        if not self.run_histories:
            return "This is a new operator that has not been evaluated yet. Your goal is to create a strong initial version."

        # Average the stats across all runs to create a single, clear time-series
        num_runs = len(self.run_histories)
        # Ensure all runs have the same number of generations
        if not all(len(run) == len(self.run_histories[0]) for run in self.run_histories):
             return "Error: Inconsistent history data."
        num_gens = len(self.run_histories[0])

        avg_history = [{"best": 0.0, "avg": 0.0, "worst": 0.0} for _ in range(num_gens)]

        for run in self.run_histories:
            for i in range(num_gens):
                avg_history[i]["best"] += run[i]["best"]
                avg_history[i]["avg"] += run[i]["avg"]
                avg_history[i]["worst"] += run[i]["worst"]

        summary_lines = [
            f"This operator's performance has been recorded over **{num_runs}** separate runs.",
            "The table below shows the fitness dynamics, **averaged across all runs**.",
            "Fitness is based on the negative loss on training batches (higher is better).\n",
            "| Gen | Best Fitness | Avg Fitness  | Worst Fitness | Spread (Diversity) |",
            "|:---:|:------------:|:------------:|:-------------:|:------------------:|",
        ]

        for i in range(num_gens):
            gen_stats = avg_history[i]
            avg_best = gen_stats['best'] / num_runs
            avg_avg = gen_stats['avg'] / num_runs
            avg_worst = gen_stats['worst'] / num_runs
            spread = avg_best - avg_worst
            summary_lines.append(f"| {i:^3} | {avg_best:12.4f} | {avg_avg:12.4f} | {avg_worst:13.4f} | {spread:18.4f} |")

        summary_lines.extend([
            "\n**Analysis Hints for Your Evolution:**",
            "- **Rate of Improvement:** Analyze the slope of the `Best Fitness` column. A steep, consistent increase is ideal.",
            "- **Population Diversity:** The `Spread (Best-Worst)` column is a proxy for diversity. If it collapses to near-zero too quickly, the population has prematurely converged, and you should consider changes that increase exploration (e.g., higher mutation, different selection).",
            "- **Stability:** Smooth, predictable improvements indicate a stable operator. Jagged or erratic values might suggest the operator is too chaotic."
        ])

        return "\n".join(summary_lines)

    def __str__(self):
        """Provides a compact summary for console logging."""
        if not self.run_histories:
            return f"Fitness: Not yet evaluated\n--- Operator Code ---\n{self.evolution_operator_code}"
        return (f"Overall Fitness (Avg Final Best): {self.fitness:.4f}\n"
                f"Evaluated over {len(self.run_histories)} runs.\n"
                f"--- Operator Code ---\n{self.evolution_operator_code}")


def selection(population, tournament_size=3):
    """Selects a parent from the population using tournament selection based on its primary fitness score."""
    if len(population) < tournament_size:
        return random.choice(population)
    tournament = random.sample(population, tournament_size)
    # The 'fitness' attribute now represents consistent high performance
    tournament.sort(key=lambda x: x.fitness, reverse=True)
    return tournament[0]

def llm_evolve_operator(operator_individual, clients):
    """Uses a randomly selected LLM client to evolve the evolutionary operator, providing detailed generational feedback."""
    if not clients:
        print("Warning: No LLM clients configured. Returning original operator.")
        return OperatorIndividual(operator_individual.evolution_operator_code)

    client = random.choice(clients)
    client_name = client.__class__.__module__.split('.')[0]

    # ✨ NEW: Generate the detailed performance summary ✨
    performance_feedback = operator_individual.get_performance_summary_text()

    prompt = f"""
You are an expert in genetic algorithms. Your task is to evolve a Python function that acts as a holistic evolutionary operator.
This function, `generate_next_population`, is the complete engine of a genetic algorithm, responsible for selection, crossover, and mutation to create the next generation of neural networks.

**Performance Feedback:**
{performance_feedback}

Based on these detailed generational dynamics, your goal is to generate a new, improved version of the operator code.
- If the operator shows **good, stable improvement**, consider a subtle refinement.
- If the operator **stagnates or converges too fast** (low spread), consider changes that increase exploration or diversity.
- If the operator is **unstable or performs poorly**, a more radical change to the evolutionary strategy might be needed.

The function signature MUST BE `def generate_next_population(current_population, fitness_scores, device, torch, SimpleNet):`.
It must return a new list of models of the same size as the input population.

**Current Operator Code:**
```python
{operator_individual.evolution_operator_code}
```
Return only the Python code block for the new, evolved function. Do not include explanations or markdown formatting.
"""
    try:
        print(f"--- Calling {client_name.capitalize()} API to evolve operator (with generational feedback) ---")
        if client_name == 'google':
            # Updated model name for best performance
            model = genai.GenerativeModel('gemini-2.5-pro')
            response = model.generate_content(prompt)
            evolved_code = response.text
        elif client_name == 'openai':
            response = client.chat.completions.create(
                model="gpt-5", # Use a strong model for this complex task
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that only returns Python code."},
                    {"role": "user", "content": prompt}
                ]
            )
            evolved_code = response.choices[0].message.content
    except Exception as e:
        print(f"Warning: API call to {client_name.capitalize()} failed: {e}. Returning original operator.")

    return OperatorIndividual(evolved_code)

def repair_operator_with_llm(faulty_code, error_trace, clients):
    """
    Attempts to repair a faulty evolutionary operator using an LLM.
    """
    if not clients:
        print("Warning: No LLM clients configured. Cannot repair operator.")
        return faulty_code

    client = random.choice(clients)
    client_name = client.__class__.__module__.split('.')[0]

    prompt = f"""
You are an expert Python programmer specializing in genetic algorithms. The following Python code for a `generate_next_population` function has failed with an error.

Your task is to analyze the code and the stack trace to identify the bug and provide a corrected version of the function.

**Faulty Code:**
```python
{faulty_code}
```

**Error Stack Trace:**
```
{error_trace}
```

Please provide only the corrected Python code block for the function. Do not include explanations or markdown formatting.
"""

    try:
        print(f"--- Calling {client_name.capitalize()} API to repair operator ---")
        if client_name == 'google':
            model = genai.GenerativeModel('gemini-2.5-pro')
            response = model.generate_content(prompt)
            repaired_code = response.text
        elif client_name == 'openai':
            response = client.chat.completions.create(
                model="gpt-5",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that only returns Python code."},
                    {"role": "user", "content": prompt}
                ]
            )
            repaired_code = response.choices[0].message.content

        return repaired_code
    except Exception as e:
        print(f"Warning: API call to {client_name.capitalize()} for repair failed: {e}. Returning original faulty code.")
        return faulty_code

def create_initial_operator_population(size, clients):
    """
    Creates a diverse first generation of operators.
    It starts with one seed function and uses the LLM to generate variations for the rest.
    """
    print(f"Creating a diverse initial population of size {size} using the LLM...")

    # The seed function remains the same standard GA
    initial_evolution_code = """

def generate_next_population(current_population, fitness_scores, device, torch, SimpleNet):
    '''
    A standard Genetic Algorithm implementation for generating the next population.
    It uses elitism, tournament selection, uniform crossover, and reset mutation.
    '''
    POPULATION_SIZE = len(current_population)
    ELITISM_RATE = 0.05
    TOURNAMENT_SIZE = 5

    # --- Helper: Crossover ---
    def crossover(parent1, parent2, device):
        child = SimpleNet().to(device)
        parent1_dict = parent1.state_dict()
        parent2_dict = parent2.state_dict()
        child_dict = child.state_dict()
        for key in parent1_dict.keys():
            mask = torch.randint(0, 2, size=parent1_dict[key].shape, device=device).float()
            child_dict[key] = (parent1_dict[key] * mask) + (parent2_dict[key] * (1 - mask))
        child.load_state_dict(child_dict)
        return child

    # --- Helper: Mutate ---
    def mutate(model, device):
        MUTATION_RESET_PROB = 0.00001
        with torch.no_grad():
            for param in model.parameters():
                mask = torch.rand_like(param.data) < MUTATION_RESET_PROB
                new_weights = torch.randn_like(param.data)
                param.data[mask] = new_weights[mask]
        return model

    # --- Helper: Tournament Selection ---
    def tournament_selection(population, scores):
        tournament_indices = torch.randint(0, len(population), (TOURNAMENT_SIZE,))
        winner_idx = tournament_indices[torch.argmax(scores[tournament_indices])]
        return population[winner_idx]

    # --- Main Generation Logic ---
    sorted_indices = torch.argsort(fitness_scores, descending=True)
    num_elite = int(POPULATION_SIZE * ELITISM_RATE)
    new_population = [current_population[i] for i in sorted_indices[:num_elite]]

    num_children_to_create = POPULATION_SIZE - num_elite
    for _ in range(num_children_to_create):
        parent1 = tournament_selection(current_population, fitness_scores)
        parent2 = tournament_selection(current_population, fitness_scores)
        child = crossover(parent1, parent2, device)
        child = mutate(child, device)
        new_population.append(child)

    return new_population
"""
    # Create the first individual from the seed code
    seed_individual = OperatorIndividual(initial_evolution_code)

    # Initialize the population with the seed
    population = [seed_individual]

    # Use the LLM to generate the rest of the initial population for diversity
    if size > 1:
        print(f"Generating {size - 1} initial variations from the seed operator...")
        # A progress bar is helpful here since this involves multiple API calls
        for _ in tqdm(range(size - 1), desc="Bootstrapping Initial Population"):
            evolved_individual = llm_evolve_operator(seed_individual, clients)
            population.append(evolved_individual)

    print("Initial population created.")
    return population

def selection(population, tournament_size=3):
    """Selects a parent from the population using tournament selection."""
    if len(population) < tournament_size:
        return random.choice(population)
    tournament = random.sample(population, tournament_size)
    tournament.sort(key=lambda x: x.fitness, reverse=True)
    return tournament[0]


# @title 6. Fitness Evaluation: Running the Inner GA
def run_inner_ga_for_fitness(operator_individual, train_loader, test_loader,clients):
    """
    Evaluates an OperatorIndividual by running a GA using its evolved operator.
    This version captures detailed per-generation statistics (best, avg, worst fitness)
    for each run to analyze the operator's behavior over time.
    """
    # --- Parameters ---
    NUM_EVAL_RUNS = 3 # Run the inner GA 3 times to average out randomness
    # --- Inner GA Parameters ---
    POPULATION_SIZE = 50
    GENERATIONS = 10
    BATCHES_PER_GENERATION = 5
    MAX_REPAIR_ATTEMPTS = 1
    for attempt in range(MAX_REPAIR_ATTEMPTS + 1):
        try:
            local_scope = {}
            exec(operator_individual.evolution_operator_code, {}, local_scope)
            generate_next_population_fn = local_scope['generate_next_population']
            # If exec is successful, break the loop
            break
        except Exception as e:
            if attempt < MAX_REPAIR_ATTEMPTS:
                print(f"Error executing evolved code. Attempting LLM repair ({attempt + 1}/{MAX_REPAIR_ATTEMPTS}).")
                error_trace = traceback.format_exc()
                repaired_code = repair_operator_with_llm(operator_individual.evolution_operator_code, error_trace, clients)
                operator_individual.evolution_operator_code = repaired_code
            else:
                print(f"Error executing evolved code after repair attempts: {e}")
                return None # Return None on final failure

    all_runs_histories = []
    print(f"Starting {NUM_EVAL_RUNS} detailed evaluation runs for this operator...")

    for run_num in range(NUM_EVAL_RUNS):
        print(f"  > Run {run_num + 1}/{NUM_EVAL_RUNS}...")
        # --- Inner GA Execution ---
        loss_fn = nn.CrossEntropyLoss()
        population = [SimpleNet().to(DEVICE) for _ in range(POPULATION_SIZE)]
        streams = [torch.cuda.Stream() for _ in range(POPULATION_SIZE)] if torch.cuda.is_available() else []
        train_iterator = iter(itertools.cycle(train_loader))

        run_history = []

        for generation in range(GENERATIONS):
            fitness_scores = torch.zeros(POPULATION_SIZE, device=DEVICE)
            for _ in range(BATCHES_PER_GENERATION):
                images, labels = next(train_iterator)
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                with torch.no_grad():
                    if torch.cuda.is_available():
                        for i, model in enumerate(population):
                            with torch.cuda.stream(streams[i]):
                                outputs = model(images)
                                loss = loss_fn(outputs, labels)
                                fitness_scores[i] += -loss # Fitness is negative loss
                    else: # CPU fallback
                         for i, model in enumerate(population):
                            outputs = model(images)
                            loss = loss_fn(outputs, labels)
                            fitness_scores[i] += -loss

            if torch.cuda.is_available():
                torch.cuda.synchronize()

            # ✨ NEW: Track detailed stats for this generation ✨
            best_fitness_gen = fitness_scores.max().item()
            worst_fitness_gen = fitness_scores.min().item()
            avg_fitness_gen = fitness_scores.mean().item()
            run_history.append({
                "generation": generation,
                "best": best_fitness_gen,
                "avg": avg_fitness_gen,
                "worst": worst_fitness_gen
            })

            # --- Evolve the population using the LLM's operator ---
            population = generate_next_population_fn(population, fitness_scores, DEVICE, torch, SimpleNet)

        all_runs_histories.append(run_history)

        # We calculate final test accuracy to report progress, but it is NOT the primary fitness metric.
        # The generational dynamics (run_history) are the key feedback for the LLM.
        best_model = population[torch.argmax(fitness_scores)]
        final_acc = evaluate_nn_model(best_model, test_loader)
        print(f"    Run {run_num + 1} finished. Final test accuracy of best model: {final_acc:.2f}%")

    # The detailed history is now the primary output of the evaluation
    return all_runs_histories


# @title 7. Run the Main Meta-Evolutionary Loop
def main():
    """The main function to run the outer (meta) EA."""
    if not llm_clients:
        print("Fatal: No LLM clients were configured. Please add API keys to Colab secrets and restart the runtime.")
        return

    # --- Meta-EA Parameters ---
    META_POPULATION_SIZE = 6
    META_GENERATIONS = 4
    MUTATION_RATE = 0.5

    print(f"Using device: {DEVICE}")
    if not torch.cuda.is_available():
        print("Warning: CUDA not available. This script is very slow on CPU.")

    train_loader, test_loader = get_data_loaders()

    operator_population = create_initial_operator_population(META_POPULATION_SIZE, llm_clients)

    for gen in range(META_GENERATIONS):
        print(f"\n{'='*25} META-GENERATION {gen + 1}/{META_GENERATIONS} {'='*25}")
        print("Evolving the `generate_next_population` operator...")

        for i, individual in enumerate(operator_population):
                # Only evaluate individuals that have no history
                if not individual.run_histories:
                    print(f"\n--- Evaluating Operator Individual {i+1}/{META_POPULATION_SIZE} ---")
                    display(Markdown(f"```python\n{individual.evolution_operator_code}\n```"))
                    # The function now returns a list of detailed run histories
                    new_histories = run_inner_ga_for_fitness(individual, train_loader, test_loader, llm_clients)

                    if new_histories is None:
                        # Assign negative infinity fitness on failure
                        individual.fitness = -float('inf')
                        print(f"Finished evaluation. Operator {i+1} failed and was assigned a fitness of -inf.")
                    else:
                        # Use the new method to update the individual's history and fitness
                        individual.update_history(new_histories)
                        print(f"Finished evaluation. Operator {i+1} primary fitness: {individual.fitness:.4f}")

        # Sort by the primary fitness metric for elitism
        operator_population.sort(key=lambda x: x.fitness, reverse=True)

        print(f"\n--- Meta-Generation {gen+1} Results ---")
        best_op = operator_population[0]
        print(f"Best Operator Fitness (Avg Final Best): {best_op.fitness:.4f}")
        print("Best Performing Operator's Code:")
        display(Markdown(f"```python\n{best_op.evolution_operator_code}\n```"))
        print("Best Operator's Performance Summary:")
        display(Markdown(best_op.get_performance_summary_text()))


        next_generation = []
        next_generation.append(best_op) # Elitism

        while len(next_generation) < META_POPULATION_SIZE:
            parent = selection(operator_population)
            if random.random() < MUTATION_RATE:
                child = llm_evolve_operator(parent, llm_clients)
            else:
                # If not mutating, create a fresh copy with the parent's history
                child = OperatorIndividual(parent.evolution_operator_code)
                child.run_histories = parent.run_histories[:] # Copy history
                child.fitness = parent.fitness
            next_generation.append(child)

        operator_population = next_generation

    print("\nMeta-Evolution finished!")
    print("Final Best Performing Operator:")
    display(Markdown(str(operator_population[0])))

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.0 MB/s eta 0:00:00
✅ Successfully configured and added Gemini client.

--- Available Gemini Models (for 'generateContent') ---
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemi

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 509kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.70MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.25MB/s]

Creating a diverse initial population of size 6 using the LLM...
Generating 5 initial variations from the seed operator...


Bootstrapping Initial Population:   0%|          | 0/5 [00:00<?, ?it/s]

--- Calling Openai API to evolve operator (with generational feedback) ---
--- Calling Openai API to evolve operator (with generational feedback) ---
--- Calling Google API to evolve operator (with generational feedback) ---
--- Calling Openai API to evolve operator (with generational feedback) ---
--- Calling Openai API to evolve operator (with generational feedback) ---
Initial population created.

========================= META-GENERATION 1/4 =========================
Evolving the `generate_next_population` operator...

--- Evaluating Operator Individual 1/6 ---


...

```python
def generate_next_population(current_population, fitness_scores, device, torch, SimpleNet):
    """
    Holistic evolutionary operator with adaptive diversity control.
    - Elitism with cloning
    - Adaptive tournament size
    - Dynamic crossover strategy (Uniform / BLX-alpha)
    - Adaptive Gaussian and reset mutation
    - Immigrants injection when diversity is low
    """
    POP_SIZE = len(current_population)

    # --- Prepare fitness tensor ---
    scores = fitness_scores
    if not torch.is_tensor(scores):
        scores = torch.tensor(scores, dtype=torch.float32, device=device)
    else:
        scores = scores.to(device=device, dtype=torch.float32)

    # Guard for degenerate populations
    if POP_SIZE == 0:
        return []

    # --- Diversity diagnostics (coefficient of variation) ---
    mean = torch.mean(scores)
    std = torch.std(scores)
    cv = std / (torch.abs(mean) + 1e-8)

    low_diversity = cv.item() < 0.05
    unstable = cv.item() > 0.5

    # --- Adaptive hyperparameters ---
    if low_diversity:
        ELITISM_RATE = 0.08
        TOURNAMENT_SIZE = 3
        GAUSS_MUT_PROB = 0.02
        GAUSS_SIGMA = 0.10
        RESET_MUT_PROB = 5e-4
        IMMIGRANTS_RATE = 0.05
        CROSSOVER_MODE = "blx"  # BLX-alpha crossover to expand search
        BLX_ALPHA = 0.20
    elif unstable:
        ELITISM_RATE = 0.07
        TOURNAMENT_SIZE = 7
        GAUSS_MUT_PROB = 0.006
        GAUSS_SIGMA = 0.03
        RESET_MUT_PROB = 1e-5
        IMMIGRANTS_RATE = 0.01
        CROSSOVER_MODE = "uniform"  # stabilize around good schemas
        BLX_ALPHA = 0.10
    else:
        ELITISM_RATE = 0.10
        TOURNAMENT_SIZE = 5
        GAUSS_MUT_PROB = 0.01
        GAUSS_SIGMA = 0.05
        RESET_MUT_PROB = 1e-4
        IMMIGRANTS_RATE = 0.02
        CROSSOVER_MODE = "mixed"  # combine uniform and arithmetic

    num_elite = max(1, int(POP_SIZE * ELITISM_RATE))
    num_immigrants = max(0, int(POP_SIZE * IMMIGRANTS_RATE))
    num_children = POP_SIZE - num_elite - num_immigrants

    # --- Utilities ---
    def clone_model(model):
        m = SimpleNet().to(device)
        m.load_state_dict(model.state_dict())
        return m

    def blx_alpha_tensor(p1, p2, alpha):
        low = torch.minimum(p1, p2)
        high = torch.maximum(p1, p2)
        span = high - low
        minv = low - alpha * span
        maxv = high + alpha * span
        return minv + (maxv - minv) * torch.rand_like(p1)

    def crossover(parent1, parent2):
        child = SimpleNet().to(device)
        p1 = parent1.state_dict()
        p2 = parent2.state_dict()
        c = child.state_dict()

        with torch.no_grad():
            for k in c.keys():
                t1 = p1[k]
                t2 = p2[k]
                if CROSSOVER_MODE == "uniform":
                    mask = (torch.rand_like(t1) < 0.5).to(t1.dtype)
                    c[k] = t1 * mask + t2 * (1.0 - mask)
                elif CROSSOVER_MODE == "blx":
                    c[k] = blx_alpha_tensor(t1, t2, BLX_ALPHA)
                else:  # mixed
                    # 50% uniform mask, 50% arithmetic blend with per-element alpha in [0.25, 0.75]
                    if torch.rand((), device=device) < 0.5:
                        mask = (torch.rand_like(t1) < 0.5).to(t1.dtype)
                        c[k] = t1 * mask + t2 * (1.0 - mask)
                    else:
                        alpha = 0.25 + 0.5 * torch.rand_like(t1)
                        c[k] = alpha * t1 + (1.0 - alpha) * t2
        child.load_state_dict(c)
        return child

    def mutate(model, gauss_prob=GAUSS_MUT_PROB, gauss_sigma=GAUSS_SIGMA, reset_prob=RESET_MUT_PROB):
        with torch.no_grad():
            for p in model.parameters():
                if gauss_prob > 0.0:
                    gmask = torch.rand_like(p) < gauss_prob
                    noise = torch.randn_like(p) * gauss_sigma
                    p[gmask] = p[gmask] + noise[gmask]
                if reset_prob > 0.0:
                    rmask = torch.rand_like(p) < reset_prob
                    new_vals = torch.randn_like(p)
                    p[rmask] = new_vals[rmask]
        return model

    def tournament_selection(population, scores_tensor, size=TOURNAMENT_SIZE):
        idxs = torch.randint(0, len(population), (size,), device=device)
        sub_scores = scores_tensor[idxs]
        winner_idx = int(idxs[torch.argmax(sub_scores)].item())
        return population[winner_idx]

    # --- Build next population ---
    sorted_idx = torch.argsort(scores, descending=True)
    elites = []
    for i in range(num_elite):
        elites.append(clone_model(current_population[int(sorted_idx[i].item())]))

    new_population = list(elites)

    # Immigrants (fresh random individuals) for diversity
    for _ in range(num_immigrants):
        new_population.append(SimpleNet().to(device))

    # Children via selection, crossover, mutation
    for _ in range(num_children):
        p1 = tournament_selection(current_population, scores)
        # ensure different parents where possible
        p2 = p1
        attempts = 0
        while p2 is p1 and attempts < 3:
            p2 = tournament_selection(current_population, scores)
            attempts += 1

        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)

    # Safety: if rounding errors occur, adjust size
    if len(new_population) > POP_SIZE:
        new_population = new_population[:POP_SIZE]
    elif len(new_population) < POP_SIZE:
        # Fill up with additional mutated clones of elites
        fill_needed = POP_SIZE - len(new_population)
        for i in range(fill_needed):
            extra = clone_model(current_population[int(sorted_idx[i % len(sorted_idx)].item())])
            new_population.append(mutate(extra))

    return new_population
```

Best Operator's Performance Summary:


This operator's performance has been recorded over **3** separate runs.
The table below shows the fitness dynamics, **averaged across all runs**.
Fitness is based on the negative loss on training batches (higher is better).

| Gen | Best Fitness | Avg Fitness  | Worst Fitness | Spread (Diversity) |
|:---:|:------------:|:------------:|:-------------:|:------------------:|
|  0  |     -11.4630 |     -11.5397 |      -11.6121 |             0.1491 |
|  1  |     -11.4313 |     -11.5767 |      -12.1839 |             0.7526 |
|  2  |     -11.4298 |     -11.5916 |      -11.9507 |             0.5209 |
|  3  |     -11.3812 |     -11.6047 |      -11.9569 |             0.5757 |
|  4  |     -11.3890 |     -11.6302 |      -12.2664 |             0.8774 |
|  5  |     -11.3751 |     -11.6719 |      -13.6228 |             2.2477 |
|  6  |     -11.3837 |     -11.6159 |      -12.1176 |             0.7339 |
|  7  |     -11.3677 |     -11.6666 |      -12.3902 |             1.0226 |
|  8  |     -11.3426 |     -11.6645 |      -12.4404 |             1.0978 |
|  9  |     -11.3403 |     -11.6956 |      -13.2220 |             1.8817 |

**Analysis Hints for Your Evolution:**
- **Rate of Improvement:** Analyze the slope of the `Best Fitness` column. A steep, consistent increase is ideal.
- **Population Diversity:** The `Spread (Best-Worst)` column is a proxy for diversity. If it collapses to near-zero too quickly, the population has prematurely converged, and you should consider changes that increase exploration (e.g., higher mutation, different selection).
- **Stability:** Smooth, predictable improvements indicate a stable operator. Jagged or erratic values might suggest the operator is too chaotic.

--- Calling Google API to evolve operator (with generational feedback) ---
--- Calling Openai API to evolve operator (with generational feedback) ---

Meta-Evolution finished!
Final Best Performing Operator:


Overall Fitness (Avg Final Best): -11.3403
Evaluated over 3 runs.
--- Operator Code ---
def generate_next_population(current_population, fitness_scores, device, torch, SimpleNet):
    """
    Holistic evolutionary operator with adaptive diversity control.
    - Elitism with cloning
    - Adaptive tournament size
    - Dynamic crossover strategy (Uniform / BLX-alpha)
    - Adaptive Gaussian and reset mutation
    - Immigrants injection when diversity is low
    """
    POP_SIZE = len(current_population)

    # --- Prepare fitness tensor ---
    scores = fitness_scores
    if not torch.is_tensor(scores):
        scores = torch.tensor(scores, dtype=torch.float32, device=device)
    else:
        scores = scores.to(device=device, dtype=torch.float32)

    # Guard for degenerate populations
    if POP_SIZE == 0:
        return []

    # --- Diversity diagnostics (coefficient of variation) ---
    mean = torch.mean(scores)
    std = torch.std(scores)
    cv = std / (torch.abs(mean) + 1e-8)

    low_diversity = cv.item() < 0.05
    unstable = cv.item() > 0.5

    # --- Adaptive hyperparameters ---
    if low_diversity:
        ELITISM_RATE = 0.08
        TOURNAMENT_SIZE = 3
        GAUSS_MUT_PROB = 0.02
        GAUSS_SIGMA = 0.10
        RESET_MUT_PROB = 5e-4
        IMMIGRANTS_RATE = 0.05
        CROSSOVER_MODE = "blx"  # BLX-alpha crossover to expand search
        BLX_ALPHA = 0.20
    elif unstable:
        ELITISM_RATE = 0.07
        TOURNAMENT_SIZE = 7
        GAUSS_MUT_PROB = 0.006
        GAUSS_SIGMA = 0.03
        RESET_MUT_PROB = 1e-5
        IMMIGRANTS_RATE = 0.01
        CROSSOVER_MODE = "uniform"  # stabilize around good schemas
        BLX_ALPHA = 0.10
    else:
        ELITISM_RATE = 0.10
        TOURNAMENT_SIZE = 5
        GAUSS_MUT_PROB = 0.01
        GAUSS_SIGMA = 0.05
        RESET_MUT_PROB = 1e-4
        IMMIGRANTS_RATE = 0.02
        CROSSOVER_MODE = "mixed"  # combine uniform and arithmetic

    num_elite = max(1, int(POP_SIZE * ELITISM_RATE))
    num_immigrants = max(0, int(POP_SIZE * IMMIGRANTS_RATE))
    num_children = POP_SIZE - num_elite - num_immigrants

    # --- Utilities ---
    def clone_model(model):
        m = SimpleNet().to(device)
        m.load_state_dict(model.state_dict())
        return m

    def blx_alpha_tensor(p1, p2, alpha):
        low = torch.minimum(p1, p2)
        high = torch.maximum(p1, p2)
        span = high - low
        minv = low - alpha * span
        maxv = high + alpha * span
        return minv + (maxv - minv) * torch.rand_like(p1)

    def crossover(parent1, parent2):
        child = SimpleNet().to(device)
        p1 = parent1.state_dict()
        p2 = parent2.state_dict()
        c = child.state_dict()

        with torch.no_grad():
            for k in c.keys():
                t1 = p1[k]
                t2 = p2[k]
                if CROSSOVER_MODE == "uniform":
                    mask = (torch.rand_like(t1) < 0.5).to(t1.dtype)
                    c[k] = t1 * mask + t2 * (1.0 - mask)
                elif CROSSOVER_MODE == "blx":
                    c[k] = blx_alpha_tensor(t1, t2, BLX_ALPHA)
                else:  # mixed
                    # 50% uniform mask, 50% arithmetic blend with per-element alpha in [0.25, 0.75]
                    if torch.rand((), device=device) < 0.5:
                        mask = (torch.rand_like(t1) < 0.5).to(t1.dtype)
                        c[k] = t1 * mask + t2 * (1.0 - mask)
                    else:
                        alpha = 0.25 + 0.5 * torch.rand_like(t1)
                        c[k] = alpha * t1 + (1.0 - alpha) * t2
        child.load_state_dict(c)
        return child

    def mutate(model, gauss_prob=GAUSS_MUT_PROB, gauss_sigma=GAUSS_SIGMA, reset_prob=RESET_MUT_PROB):
        with torch.no_grad():
            for p in model.parameters():
                if gauss_prob > 0.0:
                    gmask = torch.rand_like(p) < gauss_prob
                    noise = torch.randn_like(p) * gauss_sigma
                    p[gmask] = p[gmask] + noise[gmask]
                if reset_prob > 0.0:
                    rmask = torch.rand_like(p) < reset_prob
                    new_vals = torch.randn_like(p)
                    p[rmask] = new_vals[rmask]
        return model

    def tournament_selection(population, scores_tensor, size=TOURNAMENT_SIZE):
        idxs = torch.randint(0, len(population), (size,), device=device)
        sub_scores = scores_tensor[idxs]
        winner_idx = int(idxs[torch.argmax(sub_scores)].item())
        return population[winner_idx]

    # --- Build next population ---
    sorted_idx = torch.argsort(scores, descending=True)
    elites = []
    for i in range(num_elite):
        elites.append(clone_model(current_population[int(sorted_idx[i].item())]))

    new_population = list(elites)

    # Immigrants (fresh random individuals) for diversity
    for _ in range(num_immigrants):
        new_population.append(SimpleNet().to(device))

    # Children via selection, crossover, mutation
    for _ in range(num_children):
        p1 = tournament_selection(current_population, scores)
        # ensure different parents where possible
        p2 = p1
        attempts = 0
        while p2 is p1 and attempts < 3:
            p2 = tournament_selection(current_population, scores)
            attempts += 1

        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)

    # Safety: if rounding errors occur, adjust size
    if len(new_population) > POP_SIZE:
        new_population = new_population[:POP_SIZE]
    elif len(new_population) < POP_SIZE:
        # Fill up with additional mutated clones of elites
        fill_needed = POP_SIZE - len(new_population)
        for i in range(fill_needed):
            extra = clone_model(current_population[int(sorted_idx[i % len(sorted_idx)].item())])
            new_population.append(mutate(extra))

    return new_population